# Cloudera Docker Container

Подробное руководство [здесь](https://www.cloudera.com/documentation/enterprise/5-6-x/topics/quickstart_docker_container.html).

1. Устанавливаем `Docker`
2. Скачиваем контейнер 
> docker pull cloudera/quickstart:latest 
3. Запускаем:
> docker run --hostname=quickstart.cloudera --privileged=true --rm -it cloudera/quickstart:latest  /usr/bin/docker-quickstart

# HDFS

Hadoop Distributed File System - распределенная файловая система. Запускалась на кластере ~ 4000 машин. 
- устойчива к падениям
- масштабируема 
- безопасность (права доступа и т.д.)
- написана на Java
- концепция основана на Google GFS
- на узлах кластера используются нативные файловые системы


Узлы бывают двух видов:
- `NameNode` - хранение метаинформации
- `DataNode` - хранение данных

Бывает `Secondary NameNode`.

В целом процесс получения/записи данных выглядит так:
1. Клиент обращается к `NameNode`
2. Клиент перенаправляется к соответствующему `DataNone` для  чтения/записи данных. 


![HDFS Architecture](https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-hdfs/images/hdfsarchitecture.png)

Данные реплицируются, `NameNode` постоянно общается с `DataNode` и отслеживает их состояние, перераспределяет данные. `DataNode` могут обмениваться данными напрямую.

![](https://hadoop.apache.org/docs/r1.2.1/images/hdfsdatanodes.gif)

Логически HDFS представляет из себя привычную иерархическую файловую систему. 

- файлы делятся на блоки (по умолячанию 128Мб)
- каждый блок файла копируется по узлам сети (по умолчанию на 3 уззла)
- `NameNode` отслеживает каждый состояние узлов
- если узел стал недоступен, то блок автоматически до-реплицируется